# Passing information to the LLM

In [1]:
import ollama
LANGUAGE_MODEL = 'deepseek-r1:7b'

## A simple question...

...to which the model will confidently return a very wrong answer.

In [9]:
%%time

query = """Who was Shane MacGowan?"""

stream = ollama.generate(
    model=LANGUAGE_MODEL, 
    prompt=query, 
    stream=True,
    options={"temperature": 0},
)

for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>

</think>

Shane MacGowan is a Scottish singer-songwriter and actor who gained popularity in the late 1980s with his work on the band Status Quo. He played a significant role as the lead vocalist and guitarist for Status Quo, which achieved several hits during their career. Shane MacGowan passed away in 2005 after battling alcoholism. His music has left a lasting impact on many fans, and he is remembered for his contributions to the British music scene.CPU times: user 157 ms, sys: 45.7 ms, total: 203 ms
Wall time: 6.6 s


## Now passing information

The model will take time to think, and still return some incorrect information.

In [10]:
%%time

query = """Shane Patrick Lysaght MacGowan was a British-born Irish singer-songwriter and musician, 
best known as the lead vocalist and primary lyricist of Celtic punk band the Pogues.

Who was Shane MacGowan?"""

stream = ollama.generate(
    model=LANGUAGE_MODEL, 
    prompt=query, 
    stream=True,
    options={"temperature": 0},
)

for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>
Okay, so I need to figure out who Shane MacGowan is. From what I remember, he's connected with the Pogues, which I think is a British-Irish band known for their punk rock style. Shane Patrick Lysaght MacGowan sounds like his full name, but I'm not entirely sure about all those parts.

Wait, so Shane Patrick Lysaght MacGowan—does that mean he has multiple last names? Like Patrick and Lysaght are different? Or is it just a longer version of his surname? I think in the context of the Pogues, his full name might be more accurately represented as Shane Patrick MacGowan. Maybe "Lysaght" was an older name or something.

He's the lead vocalist and primary lyricist for the Pogues. The Pogues were a big deal in the British punk scene during the late 70s and early 80s. They had a lot of hits, like "The Boys Are Back in Town" and "I Still Haven't Found What I'm Looking For." Those are iconic songs that I recognize from their influence on alternative rock too.

Shane's voice was really dist

## Passing an entire Wikipedia article

Here we query an article from Wikipedia, pass it to the model, then ask it to answer the question based on this information. The `exintro` parameter means that we only request the introduction of the article.

It does much better job!

In [66]:
import urllib.request, json 
with urllib.request.urlopen("https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exlimit=max&explaintext&exintro&titles=Shane_MacGowan&redirects=") as url:
    wikipedia_data = json.load(url)
    #print(wikipedia_data)
article_text = list(wikipedia_data['query']['pages'].values())[0]['extract']

In [67]:
%%time
query = f"""This is an article from Wikipedia:

====
{article_text}
====

Based only on the above article, answer the following question:

Who was Shane MacGowan?"""

stream = ollama.generate(
    model=LANGUAGE_MODEL, 
    prompt=query, 
    stream=True,
    options={"temperature": 0},
)

for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>
Okay, so I need to figure out who Shane MacGowan is based solely on the provided Wikipedia article. Let me read through the article carefully and pick out the key points.

First, the article starts by introducing Shane Patrick Lysaght MacGowan as a British-born Irish singer-songwriter and musician. He's best known for being the lead vocalist and primary lyricist of the Celtic punk band the Pogues. That's his main claim to fame.

The article mentions that he gained recognition for his lyrics, which often focused on the Irish emigrant experience. So, his music has a strong theme related to immigration from Ireland. Additionally, he was noted for his lifestyle, particularly his heavy use of alcohol and drugs, which seems to have impacted both his personal life and public image.

His early life is briefly touched upon: born in Kent, England, to Irish parents, he spent his early years in Tipperary before moving back to England at age six. His academic journey includes attending Holm

In [68]:
%%time
query = f"""This is an article from Wikipedia:

====
{article_text}
====

Based only on the above article, answer the following question:

Which countries and cities are named in this article?"""

stream = ollama.generate(
    model=LANGUAGE_MODEL, 
    prompt=query, 
    stream=True,
    options={"temperature": 0},
)

for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>
Alright, I need to figure out which countries and cities are mentioned in the given Wikipedia article about Shane Patrick Lysaght MacGowan. Let me read through the article carefully.

First, it mentions he was born in Kent, England. So, England is definitely one of the countries listed. The city there would be Kent, but since Kent is a region within England, maybe just England as the country and perhaps the city if needed.

Next, the article talks about his early childhood in Tipperary, Ireland. So Ireland is another country mentioned, and Tipperary is a city or town in Ireland that's noted for this.

Then, when he moved back to England with his family at age six, so again England comes up, but since he was born there, maybe just listing it once is enough unless the article specifies different cities each time. But in this case, it's the same country twice, so probably just England and Tipperary.

Looking further, when discussing his move to London for school, it mentions Holme

## Parsing the whole article

In [69]:
import urllib.request, json 
with urllib.request.urlopen("https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exlimit=max&explaintext&titles=Shane_MacGowan&redirects=") as url:
    wikipedia_data = json.load(url)
    #print(wikipedia_data)
article_text = list(wikipedia_data['query']['pages'].values())[0]['extract']

In [70]:
%%time
query = f"""This is an article from Wikipedia:

====
{article_text}
====

Based only on the above article, answer the following question:

Which countries and cities are named in this article?"""

stream = ollama.generate(
    model=LANGUAGE_MODEL, 
    prompt=query, 
    stream=True,
    options={"temperature": 0},
)

for chunk in stream:
  print(chunk['response'], end='', flush=True)

<think>
Alright, I need to figure out which countries and cities are mentioned in the provided article about Shane MacGowan. Let me go through the text step by step.

First, looking at the "Early life" section, it mentions that Shane grew up in County Wexford, Ireland. So, County Wexford is a country (specifically a county) within Ireland. The city mentioned here is Cavan, which I know is a city in County Wexford.

Next, under the "Autobiography and biographies" section, there's a mention of an album called "The Nips/Nipple Erectors" released in 1987/2003. That doesn't add any new countries or cities since it's just an album title.

Moving on to the "Discography" section, I see albums like "Bops, Babes, Booze & Bovver," which is a compilation from 1987/2003. Again, no new country info there.

In the "Albums" section under "With the Pogues," the first album listed is "Red Roses for Me" released in October 1984. No countries or cities mentioned here.

Looking further down, under "Albums"

This time, despite having more information, the model gives less complete and partly incorrect answers!